In [15]:
import os
import pandas as pd
from autograd import grad
import autograd.numpy as np
import numpy as np1
import numpy.matlib
from numpy.linalg import inv
from numpy import linalg as LA
from numpy.matlib import repmat

In [16]:
import sys
sys.path.append('../../Python/')
from Preprocessing import standardize
from Evaluation import split_train_test_ma, train_model, test_model
from Databases import get_databases_path, get_database

In [17]:
databases_path = '../../Databases/Sinteticas'
paths = get_databases_path(databases_path)

In [18]:
# Obtain the attributes and labels
attr_df = get_database(paths[1], 'database_attr.csv')
labels_df = get_database(paths[1], 'database_labels.csv')

In [19]:
# Preprocessing the attributes
standardized_attr_df = standardize(attr_df)

In [20]:
# Split in Training and Test Sets
X_train, X_test, y_train, y_test = split_train_test_ma(standardized_attr_df, labels_df)

In [23]:
def sigmoid(a):
    return 1./(1 + np.exp(-a))

def grad_cost_function(w, X, Y, mui):
    R = Y.shape[1] # Number of annotators.
    D = X.shape[1] # Input space dimension
    N = X.shape[0] # Number of samples.
    
    sizew1 = D
        
    W1 = w.reshape(sizew1,1)
    pi = sigmoid(np.dot(X, W1))
    pi = np1.matlib.repmat(pi, 1, R)
    aux = np.multiply(mui, Y) - np.multiply(mui, pi)
    aux2 = -np.sum(np.dot(X.transpose(), aux), axis=1)
    g = aux2.reshape(1, sizew1)
    return g
     
def optimizer(X, Y, w_0, mui):
    params = w_0
    epsilon = 0.001
    normGrad = 300
    i=0
    while normGrad > 0.1 and i<300:
        gr = grad_cost_function(params, X, Y, mui)
        params = params - epsilon * gr
        normGrad = LA.norm(gr)
        i+=1
    return params, normGrad
    
def traing_mae(X, Y):
    X.loc[:,'b'] = np.zeros((X.shape[0],1))
    X = X.values
    Y = Y.values
    R = Y.shape[1] # Number of annotators.
    D = X.shape[1] # Input space dimension
    N = X.shape[0] # Number of samples.
    
    
    sizew1 = D
    sizew2 = R;
    sizeW = sizew1 + sizew2
    
    w = np.random.randn(1, sizeW)
    Yhat = np.array(Y.mean(axis=1)).reshape(Y.shape[0],1)
    aux = np1.matlib.repmat(numpy.around(Yhat), 1, R) - Y
    mui = 1 - abs(aux)
    
    w1 = np.dot(inv(np.dot(X.T,X) + 1e-20*np.eye(D)), np.dot(X.T,Yhat))
    w1 = w1.reshape(1, sizew1)
    w2 = mui.sum(axis=0)/N

    w_0 = w1
    loglik1 = 10000000
    difloglik = 100
    difloglik1 = -1
    i = 0
    
    while abs(difloglik) > 0.2 and i<100:
        # M-step     
        w, Ngr = optimizer(X, Y, w_0, mui)
        W1 = w[0,0:sizew1].reshape(sizew1,1)
        w2 = mui.sum(axis=0)/N

        #E-step
        Yhat = sigmoid(np.dot(X, W1))
        pi = Yhat
        auxpi = np1.matlib.repmat(pi, 1, R)
        auxpir = np1.matlib.repmat(w2, N, 1)
        
        nummui = np.multiply(auxpir, auxpi)
        denmui = (np.multiply(auxpir, auxpi) + 0.5*(1-auxpir)) 
        mui =  np.divide(nummui, denmui)
        
        loglika = np.sum(np.multiply(np.log(auxpir), mui))
        auxb = np.sum(np.multiply(Y, mui,), axis=1).reshape(N, 1)
        loglikb = np.sum(np.multiply(np.log(pi), auxb))
        auxc = np.sum(np.multiply((1-Y), mui,), axis=1).reshape(N, 1)
        loglikc = np.sum(np.multiply(np.log((1-pi)), auxc))
        loglikd = np.sum(np.multiply(np.log((1-auxpir)), (1-mui)))
        
        loglik2 = -(loglika + loglikb + loglikc + loglikd)
        difloglik = (loglik2 - loglik1);
        loglik1 = loglik2

        w_0 = w
        i+=1
    return w1.reshape(sizew1,1) 

In [24]:
W1 = traing_mae(X_train, y_train)

Verosimilitud: 9997763.615935452
Verosimilitud: 174.03646194973908
Verosimilitud: 121.84280885209637
Verosimilitud: 2.3917324130825364
Verosimilitud: 57.89585996377036
Verosimilitud: 20.39877791635172
Verosimilitud: 40.43577594517001
Verosimilitud: 20.47429422848745
Verosimilitud: 31.704075471071974
Verosimilitud: 17.46352681406779
Verosimilitud: 26.496272573310307
Verosimilitud: 13.975466581406181
Verosimilitud: 23.515288892548597
Verosimilitud: 10.264942103751764
Verosimilitud: 22.4997708904084
Verosimilitud: 5.777513906486774
Verosimilitud: 23.99190672902455
Verosimilitud: 0.9117003864803337
Verosimilitud: 30.000691510999786
Verosimilitud: 13.285597842875859
Verosimilitud: 47.176337353169174
Verosimilitud: 42.29253233113536
Verosimilitud: 71.87407290738906
Verosimilitud: 44.28902837043097
Verosimilitud: 126.75975095960666
Verosimilitud: 114.8652731441282
Verosimilitud: 72.48343151036602
Verosimilitud: 44.066137549701125
Verosimilitud: 25.792148298080065
Verosimilitud: 10.88699696828

Verosimilitud: 2.2721065892082493
Verosimilitud: 2.2601672560209067
Verosimilitud: 2.2483038680704794
Verosimilitud: 2.236515835816874
Verosimilitud: 2.224802574770763
Verosimilitud: 2.2131635054558387
Verosimilitud: 2.201598053374539
Verosimilitud: 2.1901056489705866
Verosimilitud: 2.1786857275924945
Verosimilitud: 2.1673377294577563
Verosimilitud: 2.1560610996158402
Verosimilitud: 2.1448552879112412
Verosimilitud: 2.13371974894676
Verosimilitud: 2.122653942048146
Verosimilitud: 2.1116573312257856
Verosimilitud: 2.1007293851382087
Verosimilitud: 2.0898695770566746
Verosimilitud: 2.079077384826178
Verosimilitud: 2.068352290831797
Verosimilitud: 2.0576937819596424
Verosimilitud: 2.04710134956116
Verosimilitud: 2.0365744894179443
Verosimilitud: 2.0261127017026297
Verosimilitud: 2.0157154909455244
Verosimilitud: 2.0053823659965246
Verosimilitud: 1.9951128399904974
Verosimilitud: 1.9849064303091382
Verosimilitud: 1.9747626585490252
Verosimilitud: 1.9646810504808627
Verosimilitud: 1.9546611

In [26]:
X_test['b'] = 1.0
X_test_matrix = X_test.values
pred = sigmoid(np.dot(X_test_matrix, W1))
y_pred = [True if item >= 0.5 else False for item in pred]

/Users/codigoriginal/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [27]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
print("Global Precision:")
print(accuracy_score(y_test, y_pred))
print("\n")

print("General Report:")
print(classification_report(y_test, y_pred, target_names=['yes','no']))
print("\n")

print("Confusion Matrix: ")
matriz_confusion = confusion_matrix(y_test, y_pred)
table = pd.DataFrame(matriz_confusion)
print(table)

Global Precision:
0.7309941520467836


General Report:
              precision    recall  f1-score   support

         yes       0.80      0.76      0.78       108
          no       0.62      0.68      0.65        63

    accuracy                           0.73       171
   macro avg       0.71      0.72      0.72       171
weighted avg       0.74      0.73      0.73       171



Confusion Matrix: 
    0   1
0  82  26
1  20  43
